Importuojamos reikalingos bibliotekos

In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
import calendar
import time
from sklearn.linear_model import LinearRegression

Nuskaitomi duomenys

In [353]:
train_data = pd.read_csv("C:/Users/Paulius/Documents/GitHub/DVDA_PROJEKTAS/project/1-data/train_data.csv")
test_data = pd.read_csv("C:/Users/Paulius/Documents/GitHub/DVDA_PROJEKTAS/project/1-data/test_data.csv")

Peržiūrima nuskaitytų duomenų informacija

In [354]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 17 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   id                            1000000 non-null  int64  
 1   y                             1000000 non-null  int64  
 2   amount_current_loan           1000000 non-null  int64  
 3   term                          1000000 non-null  object 
 4   credit_score                  685667 non-null   object 
 5   loan_purpose                  1000000 non-null  object 
 6   yearly_income                 780561 non-null   float64
 7   home_ownership                1000000 non-null  object 
 8   bankruptcies                  998195 non-null   float64
 9   years_current_job             954051 non-null   float64
 10  monthly_debt                  1000000 non-null  float64
 11  years_credit_history          1000000 non-null  int64  
 12  months_since_last_delinquent 

Identifikuojami kategoriniai kintamieji bei skirtingos jų reikšmės. Taip pat patikrinama, ar kategorinio kintamojo reikšmė egzistuoja ir treniravimo, ir testavimo duomenų rinkiniuose

In [355]:
for column in train_data:
    if train_data[column].dtype == "object":
        print("Column: "+ column)
        print(train_data[column].unique())
        print("Amount of categories: " + str(len(train_data[column].unique())))
        print(set(train_data[column].unique()).difference(set(test_data[column].unique())))
        print(set(test_data[column].unique()).difference(set(train_data[column].unique())))
        print("")

Column: term
['short' 'long']
Amount of categories: 2
set()
set()

Column: credit_score
['very_good' 'good' nan 'fair']
Amount of categories: 4
set()
set()

Column: loan_purpose
['other' 'debt_consolidation' 'home_improvements' 'take_a_trip'
 'medical_bills' 'educational_expenses' 'major_purchase' 'small_business'
 'business_loan' 'buy_a_car' 'moving' 'buy_house' 'vacation' 'wedding'
 'renewable_energy']
Amount of categories: 15
{'renewable_energy'}
set()

Column: home_ownership
['rent' 'mortgage' 'own']
Amount of categories: 3
set()
set()



Galima pastebėti, kad testavimo duomenų rinkio stulpelyje loan_purpose nėra reikšmės renewable_energy, todėl visas eilutes su šia reikšme galima ištrinti iš treniravimo duomenų rinkinio

In [356]:
train_data = train_data[train_data['loan_purpose']!="renewable_energy"]

Pateikiama tolydinio tipo atributų kokybės analizė

In [357]:
train_data.describe()

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,999891.000000,999891.000000,999891.000000,7.804840e+05,998087.000000,953947.000000,999891.000000,999891.000000,470399.000000,999891.000000,999891.000000,9.998910e+05,9.998640e+05
mean,499992.410771,0.499987,316671.330151,1.344777e+06,0.119207,5.875944,18549.466006,18.095021,34.946356,11.184453,0.176171,2.938495e+05,7.367368e+05
std,288673.862734,0.500000,186860.678385,1.278992e+06,0.354673,3.625401,12090.579058,7.069086,22.097677,5.068003,0.495867,3.666039e+05,7.147263e+06
min,1.000000,0.000000,10802.000000,7.662700e+04,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,249998.500000,0.000000,174394.000000,8.257400e+05,0.000000,3.000000,10323.080000,13.000000,16.000000,8.000000,0.000000,1.133920e+05,2.700005e+05
50%,499988.000000,0.000000,269676.000000,1.148550e+06,0.000000,6.000000,16318.150000,17.000000,32.000000,10.000000,0.000000,2.105770e+05,4.599980e+05
75%,749985.500000,1.000000,435160.000000,1.605728e+06,0.000000,10.000000,24059.320000,22.000000,51.000000,14.000000,0.000000,3.674410e+05,7.673600e+05
max,1000000.000000,1.000000,789250.000000,1.655574e+08,7.000000,10.000000,435843.280000,70.000000,176.000000,76.000000,15.000000,3.287897e+07,1.539738e+09


Randamos trūkstamos reikšmės kiekviename stulpelyje

In [358]:
train_data.isnull().sum()

id                                   0
y                                    0
amount_current_loan                  0
term                                 0
credit_score                    314299
loan_purpose                         0
yearly_income                   219407
home_ownership                       0
bankruptcies                      1804
years_current_job                45944
monthly_debt                         0
years_credit_history                 0
months_since_last_delinquent    529492
open_accounts                        0
credit_problems                      0
credit_balance                       0
max_open_credit                     27
dtype: int64

In [359]:
train_data.drop(train_data[(train_data['credit_score'].isnull()) & (train_data['months_since_last_delinquent'].isnull()) & (train_data['yearly_income'].isnull())].index, inplace=True)

In [360]:
train_data.isnull().sum()

id                                   0
y                                    0
amount_current_loan                  0
term                                 0
credit_score                    194241
loan_purpose                         0
yearly_income                    99349
home_ownership                       0
bankruptcies                      1800
years_current_job                40362
monthly_debt                         0
years_credit_history                 0
months_since_last_delinquent    409434
open_accounts                        0
credit_problems                      0
credit_balance                       0
max_open_credit                     25
dtype: int64

In [361]:
train_data = train_data[train_data['max_open_credit'].notnull()]
train_data = train_data[train_data['bankruptcies'].notnull()]
train_data = train_data[train_data['years_current_job'].notnull()]

train_data['credit_score'] = train_data['credit_score'].fillna('unknown')

In [362]:
train_data.describe()

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,837713.000000,837713.000000,837713.000000,7.436140e+05,837713.000000,837713.000000,837713.000000,837713.000000,447079.000000,837713.000000,837713.000000,8.377130e+05,8.377130e+05
mean,497591.559184,0.495036,321103.597323,1.367086e+06,0.115184,5.908472,18895.988768,17.976106,34.936371,11.281723,0.174346,2.934773e+05,7.265559e+05
std,288704.305195,0.499976,187659.307664,1.301110e+06,0.350264,3.626195,12172.305339,6.824084,22.046206,5.099867,0.497226,3.683635e+05,6.438295e+06
min,1.000000,0.000000,10802.000000,7.662700e+04,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,247477.000000,0.000000,176616.000000,8.441510e+05,0.000000,3.000000,10665.270000,13.000000,16.000000,8.000000,0.000000,1.130500e+05,2.686200e+05
50%,495107.000000,0.000000,277706.000000,1.161204e+06,0.000000,6.000000,16654.450000,17.000000,32.000000,10.000000,0.000000,2.100450e+05,4.574020e+05
75%,747624.000000,1.000000,437888.000000,1.628509e+06,0.000000,10.000000,24393.530000,22.000000,51.000000,14.000000,0.000000,3.659020e+05,7.633780e+05
max,1000000.000000,1.000000,789250.000000,1.655574e+08,7.000000,10.000000,435843.280000,70.000000,176.000000,76.000000,15.000000,3.287897e+07,1.539738e+09


In [363]:
train_data.isnull().sum()

id                                   0
y                                    0
amount_current_loan                  0
term                                 0
credit_score                         0
loan_purpose                         0
yearly_income                    94099
home_ownership                       0
bankruptcies                         0
years_current_job                    0
monthly_debt                         0
years_credit_history                 0
months_since_last_delinquent    390634
open_accounts                        0
credit_problems                      0
credit_balance                       0
max_open_credit                      0
dtype: int64

In [367]:
test_data_inc_null = train_data[train_data['yearly_income'].isnull()]
test_data_inc_notnull = train_data[train_data['yearly_income'].notnull()]
train_data.dropna(inplace=True)

x_train = train_data.drop('yearly_income', axis=1)
y_train = train_data['yearly_income']

lr = LinearRegression()

lr.fit(x_train, y_train)

x_test = test_data_inc_null.drop('yearly_income', axis=1)

y_pred = lr.predict(x_test).astype(int)

test_data_inc_null['yearly_income'] = y_pred

train_data = pd.concat([test_data_inc_null, test_data_inc_notnull])

C:\Users\Paulius\AppData\Local\Temp\ipykernel_13636\1954850374.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_inc_null['yearly_income'] = y_pred


In [373]:
train_data.to_csv('C:/Users/Paulius/Documents/GitHub/DVDA_PROJEKTAS/project/1-data/train_data_new.csv', sep=',', encoding='utf-8', index=False)